In [ ]:
# 🧮 Cart-Pole System: Euler Method Simulation with SymPy + Matplotlib

# --- Imports ---
from sympy import symbols, Function, diff, simplify, lambdify, sin, cos
from sympy.physics.mechanics import dynamicsymbols
import numpy as np
import matplotlib.pyplot as plt

# --- Define symbols ---
t = symbols('t')
x = Function('x')(t)
x_dot = diff(x, t)
x_ddot = diff(x, t, t)

theta = Function('theta')(t)
theta_dot = diff(theta, t)
theta_ddot = diff(theta, t, t)

m, M, l, g, F, cx, ctheta = symbols('m M l g F cx ctheta')

# --- Kinetic and potential energy ---
# Kinetic Energy
T_cart = (1/2)*M*x_dot**2
v_pendulum = x_dot + l * cos(theta) * theta_dot  # horizontal velocity of mass
w_pendulum = l * sin(theta) * theta_dot         # vertical velocity of mass
T_pendulum = (1/2)*m * (v_pendulum**2 + w_pendulum**2)
T = simplify(T_cart + T_pendulum)

# Potential Energy
V = -m * g * l * cos(theta)

# Lagrangian
L = T - V

# --- Generalized Forces (damping + external force) ---
Q_x = F - cx * x_dot
Q_theta = -ctheta * theta_dot

# --- Euler-Lagrange Equations ---
dL_dx = diff(L, x)
dL_dx_dot = diff(L, x_dot)
d_dt_dL_dx_dot = diff(dL_dx_dot, t)
EL_x = simplify(d_dt_dL_dx_dot - dL_dx - Q_x)

dL_dtheta = diff(L, theta)
dL_dtheta_dot = diff(L, theta_dot)
d_dt_dL_dtheta_dot = diff(dL_dtheta_dot, t)
EL_theta = simplify(d_dt_dL_dtheta_dot - dL_dtheta - Q_theta)

# --- Solve for accelerations ---
sol = simplify(solve([EL_x, EL_theta], (x.diff(t, t), theta.diff(t, t))))
x_ddot_expr = sol[x.diff(t, t)]
theta_ddot_expr = sol[theta.diff(t, t)]

# --- Lambdify expressions ---
x_ddot_func = lambdify((x, x_dot, theta, theta_dot, F, m, M, l, g, cx, ctheta), x_ddot_expr)
theta_ddot_func = lambdify((x, x_dot, theta, theta_dot, F, m, M, l, g, cx, ctheta), theta_ddot_expr)


# 📐 Euler Method Simulation for Cart-Pole
# ----------------------------------------------------------
# In this section, we simulate the nonlinear cart-pole system
# using the Explicit Euler method. This numerical integrator
# updates the state step-by-step based on derivative values.
# ----------------------------------------------------------

# --- Parameters ---
params = {
    'm': 0.1,     # pendulum mass [kg]
    'M': 1.0,     # cart mass [kg]
    'l': 1.0,     # pendulum length [m]
    'g': 9.81,    # gravity [m/s^2]
    'cx': 0.1,    # cart damping
    'ctheta': 0.05 # pendulum damping
}

# --- Initial conditions ---
state = np.array([0.0, 0.0, 0.2, 0.0])  # x, x_dot, theta, theta_dot
T_total = 10.0
dt = 0.01
N = int(T_total / dt)
time = np.linspace(0, T_total, N)

# --- Storage ---
history = np.zeros((N, 4))
history[0, :] = state

# --- External force function (optional: 0 or impulse etc) ---
def external_force(t):
    return 0.0

# --- Euler Integration Loop ---
for i in range(1, N):
    x, x_dot, theta, theta_dot = state
    F_ext = external_force(time[i])

    dx2 = x_ddot_func(x, x_dot, theta, theta_dot, F_ext, **params)
    dtheta2 = theta_ddot_func(x, x_dot, theta, theta_dot, F_ext, **params)

    # Euler step
    x_dot += dx2 * dt
    x += x_dot * dt
    theta_dot += dtheta2 * dt
    theta += theta_dot * dt

    state = np.array([x, x_dot, theta, theta_dot])
    history[i, :] = state


# 📊 Plotting the Simulation Results
# ----------------------------------
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(time, history[:, 0], label='x (cart position)')
plt.plot(time, history[:, 1], label='x_dot')
plt.xlabel('Time [s]')
plt.ylabel('Cart Variables')
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(time, history[:, 2], label='theta (rad)')
plt.plot(time, history[:, 3], label='theta_dot')
plt.xlabel('Time [s]')
plt.ylabel('Pendulum Variables')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()
